# Method 1

In [ ]:
import cv2
import numpy as np
from imutils import contours

image = cv2.imread('images/2.jpeg')
original = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 500, 1000, 1)

vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,2))
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,1))
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
erode = cv2.erode(canny, vertical_kernel)
cv2.imshow('remove horizontal', erode)
dilate = cv2.dilate(erode, vertical_kernel, iterations=5)
cv2.imshow('dilate vertical', dilate)
erode = cv2.erode(dilate, horizontal_kernel, iterations=1)
cv2.imshow('remove vertical', erode)
dilate = cv2.dilate(erode, kernel, iterations=4)
cv2.imshow('dilate horizontal', dilate)

cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

digit_contours = []
for c in cnts:
    area = cv2.contourArea(c)
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.01 * peri, True)
    x,y,w,h = cv2.boundingRect(approx)
    aspect_ratio = w / float(h)

    if (aspect_ratio >= 0.4 and aspect_ratio <= 1.3):
        if area > 150:
            ROI = original[y:y+h, x:x+w]
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            digit_contours.append(c)

sorted_digit_contours = contours.sort_contours(digit_contours, method='left-to-right')[0]
contour_number = 0
for c in sorted_digit_contours:
    x,y,w,h = cv2.boundingRect(c)
    ROI = original[y:y+h, x:x+w]
    # cv2.imwrite('ROI_{}.png'.format(contour_number), ROI)
    contour_number += 1

cv2.imshow('canny', canny)
cv2.imshow('image', image)
cv2.waitKey(0)

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'


# Method 2

In [ ]:
import cv2
import numpy as np

# --- Optional: set up background subtractor (only relevant for live/video) ---
bgsub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

# Read Input image
inputImage = cv2.imread("./images/2.jpeg")
inputImageCopy = inputImage.copy()

# 1) Warp/ROI isolation would go here if you had it…
#    (we’ll just operate on the full image)

# 2) Convert to grayscale + CLAHE for contrast
gray = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
gray = clahe.apply(gray)

# 3) Adaptive Thresholding
blur = cv2.GaussianBlur(gray, (5,5), 0)
th_adapt = cv2.adaptiveThreshold(
    blur, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    blockSize=15,
    C=5
)

# 4) Canny Edge Detection (optional, helps reinforce strokes)
edges = cv2.Canny(gray, 50, 150)
edges = cv2.dilate(edges,
    cv2.getStructuringElement(cv2.MORPH_RECT, (3,3)),
    iterations=1
)

# 5) (Optional) combine with background‐subtraction mask
fgmask = bgsub.apply(inputImage)
combined = cv2.bitwise_or(th_adapt, edges)       # merge edges + threshold
combined = cv2.bitwise_and(combined, fgmask)     # ignore background motion

# 6) Morphological clean‐up
kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
kernel5 = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
clean = cv2.morphologyEx(combined, cv2.MORPH_OPEN,  kernel3, iterations=1)
clean = cv2.morphologyEx(clean,  cv2.MORPH_CLOSE, kernel5, iterations=1)

# 7) Flood-fill border (as you had) to remove edge‐touching blobs
cv2.floodFill(clean, None, (0, 0), 0)

# 8) Find external contours on the cleaned mask
contours, _ = cv2.findContours(
    clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

minArea = 5000
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if w*h < minArea:
        continue

    # square‐ify as before
    size = max(w, h)
    cx, cy = x + w//2, y + h//2
    x1, y1 = max(cx-size//2, 0), max(cy-size//2, 0)
    x2, y2 = min(x1+size, inputImage.shape[1]), min(y1+size, inputImage.shape[0])

    # draw & extract
    cv2.rectangle(inputImageCopy, (x1,y1), (x2,y2), (0,255,0), 2)
    squareROI = inputImage[y1:y2, x1:x2]

    # downscale + grayscale
    smallROI   = cv2.resize(squareROI, (32,32), interpolation=cv2.INTER_AREA)
    smallGray  = cv2.cvtColor(smallROI, cv2.COLOR_BGR2GRAY)

    # display
    cv2.imshow("Clean Mask", clean)
    cv2.imshow("Square Crop (orig)", squareROI)
    cv2.imshow("32x32 Gray",     smallGray)
    cv2.imshow("All Boxes",      inputImageCopy)
    cv2.waitKey(0)

cv2.destroyAllWindows()


2025-05-25 20:07:47.503 python[59119:155506500] +[IMKClient subclass]: chose IMKClient_Legacy
2025-05-25 20:07:47.503 python[59119:155506500] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 